In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Lasso
from utils import load_inp_outp, create_network_df, prepare_network_data, load_inp_outp_cap
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
#from efficient_kan import KAN
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pickle
import os

In [60]:
pwd = os.path.join(os.getcwd())
directory_sioux = pwd + "/data/sioux/uncongested"
directory_ema = pwd + "/data/ema/new/no_capacities"
directory_sample = pwd + "/data/custom/no_capacities"

# Linear Regression

## Part 1: fixed capacities and free-flow times

In [91]:
OD_sioux, target_flows_sioux = load_inp_outp(directory_sioux)
OD_sioux = OD_sioux[:4000]
target_flows_sioux = target_flows_sioux[:4000]
X_sioux = OD_sioux.reshape(OD_sioux.shape[0], -1)
nonzero_indices = np.nonzero(target_flows_sioux[0])
y_sioux = target_flows_sioux[:, nonzero_indices[0], nonzero_indices[1]]

OD_ema, target_flows_ema = load_inp_outp(directory_ema)
X_ema = OD_ema.reshape(OD_ema.shape[0], -1)
nonzero_indices = np.nonzero(target_flows_ema[0])
y_ema = target_flows_ema[:, nonzero_indices[0], nonzero_indices[1]]

OD_sample, target_flows_sample = load_inp_outp(directory_sample)
X_sample = OD_sample.reshape(OD_sample.shape[0], -1)
nonzero_indices = np.nonzero(target_flows_sample[0])
y_sample = target_flows_sample[:, nonzero_indices[0], nonzero_indices[1]]

In [62]:
X_sioux.shape, X_ema.shape, X_sample.shape

((5000, 576), (5000, 5476), (4390, 27556))

In [95]:
scaler_X_sioux = StandardScaler()
X_scaled_sioux = scaler_X_sioux.fit_transform(X_sioux[:4000])

scaler_y_sioux = StandardScaler()
y_scaled_sioux = scaler_y_sioux.fit_transform(y_sioux[:4000])

X_train_sioux, X_test_sioux, y_train_sioux, y_test_sioux = train_test_split(X_scaled_sioux, y_scaled_sioux, test_size=0.1, random_state=42, shuffle=True)
model_sioux = LinearRegression()
model_sioux_lasso = Lasso()
model_sioux.fit(X_train_sioux, y_train_sioux)

LinearRegression()

In [ ]:
scaler_X_ema = StandardScaler()
X_scaled_ema = scaler_X_ema.fit_transform(X_ema)

scaler_y_ema = StandardScaler()
y_scaled_ema = scaler_y_ema.fit_transform(y_ema)

X_train_ema, X_test_ema, y_train_ema, y_test_ema = train_test_split(X_scaled_ema, y_scaled_ema, test_size=0.1, random_state=42, shuffle=False)

LinearRegression()

In [ ]:
model_ema = LinearRegression()
model_ema.fit(X_train_ema, y_train_ema)

In [100]:
scaler_X_sample = StandardScaler()
X_scaled_sample = scaler_X_sample.fit_transform(X_sample)

scaler_y_sample = StandardScaler()
y_scaled_sample = scaler_y_sample.fit_transform(y_sample)

X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X_scaled_sample, y_scaled_sample, test_size=0.1, random_state=42, shuffle=False)

In [15]:
model_sample = LinearRegression()
model_sample.fit(X_train_sample[:100], y_train_sample[:100])

LinearRegression()

In [14]:
for i in [X_train_sample, X_test_sample, y_train_sample, y_test_sample]:
    print(i.shape)

(3951, 27556)
(439, 27556)
(3951, 302)
(439, 302)


In [ ]:
assert False

AssertionError: 

In [ ]:
# # nonzero_indices = np.nonzero(target_flows_sioux[0])
# # capacities_sioux = capacities_sioux[:, nonzero_indices[0], nonzero_indices[1]]
# nonzero_indices = np.nonzero(target_flows_ema[0])
# capacities_ema = capacities_ema[:, nonzero_indices[0], nonzero_indices[1]]

In [101]:
y_pred_scaled_sioux = model_sioux.predict(X_test_sioux)
y_pred_scaled_ema = model_ema.predict(X_test_ema)
# y_pred_sioux = np.minimum.reduce([scaler_y_sioux.inverse_transform(y_pred_scaled_sioux), capacities_sioux[:len(X_test_sioux)]], axis=0)
y_pred_sioux = scaler_y_sioux.inverse_transform(y_pred_scaled_sioux)
y_test_original_sioux = scaler_y_sioux.inverse_transform(y_test_sioux)
# y_pred_ema = np.minimum.reduce([scaler_y_ema.inverse_transform(y_pred_scaled_ema), capacities_ema[:len(X_test_ema)]], axis=0)
y_pred_ema = scaler_y_ema.inverse_transform(y_pred_scaled_ema)
y_test_original_ema = scaler_y_ema.inverse_transform(y_test_ema)

In [97]:
y_pred_scaled_sample = model_sample.predict(X_test_sample)
y_pred_sample = scaler_y_sample.inverse_transform(y_pred_scaled_sample)
y_test_original_sample = scaler_y_sample.inverse_transform(y_test_sample)

In [76]:
def safe_mape(y_true, y_pred):
    mask = np.abs(y_true) > 0.00001
    if np.sum(mask) == 0:
        return 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

In [77]:
def evaluate(y_true, y_pred):
    mape_list = []
    mae_list = []
    mse_list = []
    r2_list = []
    for sample in range(y_true.shape[0]):
        true_edge = y_true[sample]
        pred_edge = y_pred[sample]
        mask = true_edge != 0
        if np.sum(mask) == 0:
            continue

        true_values = true_edge[mask]
        pred_values = pred_edge[mask]

        mape_list.append(safe_mape(true_edge, pred_edge))
        mae_list.append(mean_absolute_error(true_values, pred_values))
        mse_list.append(mean_squared_error(true_values, pred_values))
        r2_list.append(r2_score(true_values, pred_values))

    mape_list = np.array(mape_list)
    metrics = {
        'MSE': np.mean(mse_list) if mse_list else np.nan,
        'RMSE': np.sqrt(np.mean(mse_list)) if mse_list else np.nan,
        'MAE': np.mean(mae_list) if mae_list else np.nan,
        'R2': np.mean(r2_list) if r2_list else np.nan,
        'MAPE': np.mean(mape_list[np.abs(mape_list) < 1]),
        'MedianAE': np.median(mae_list) if mae_list else np.nan
    }
    return metrics

In [98]:
metrics_sioux = evaluate(y_test_original_sioux, y_pred_sioux)
metrics_ema = evaluate(y_test_original_ema, y_pred_ema)
metrics_sample = evaluate(y_test_original_sample, y_pred_sample)

In [99]:
metrics_sioux

{'MSE': np.float64(35361.12423957216),
 'RMSE': np.float64(188.04553767524544),
 'MAE': np.float64(108.9934374035638),
 'R2': np.float64(0.9941667030263419),
 'MAPE': np.float64(0.045131604592863896),
 'MedianAE': np.float64(87.53060856008125)}

In [ ]:
target_flows_sioux[target_flows_sioux.nonzero()].mean()

np.float64(3828.7822581139667)

In [55]:
metrics_ema

{'MSE': np.float64(804.2135487656177),
 'RMSE': np.float64(28.358659149642772),
 'MAE': np.float64(10.97151680730338),
 'R2': np.float64(0.9995044284884946),
 'MAPE': np.float64(0.013388953400777967),
 'MedianAE': np.float64(10.413788364734412)}

In [ ]:
y_pred_sample.shape, y_sample.shape

((439, 302), (4390, 302))

In [ ]:
safe_mape(y_test_original_sample, y_pred_sample)

np.float64(0.05252362824292709)

In [ ]:
target_flows_ema[target_flows_ema.nonzero()].mean()

np.float64(1333.288433918223)

In [ ]:
metrics_sample

{'MSE': np.float64(10614898.93812243),
 'RMSE': np.float64(3258.051402007407),
 'MAE': np.float64(1664.9724220778307),
 'R2': np.float64(0.9922488022072249),
 'MAPE': np.float64(0.05252362824292709),
 'MedianAE': np.float64(1519.5797508700464)}

In [ ]:
a = np.round((np.abs(y_test_original_ema - y_pred_ema) / y_pred_ema), 2)

In [ ]:
y_pred_sample.shape, y_train_sample.shape

((439, 302), (3951, 302))

## Part 2: fixed free-flow times

In [ ]:
directory_sioux = pwd + "/data/sioux/with_capacities"
directory_ema = pwd + "/data/ema/with_capacities"

In [ ]:
sioux = create_network_df(network_name="SiouxFalls")
T_0_sioux, C_sioux = prepare_network_data(sioux)

ema = create_network_df(network_name="EMA")
T_0_sioux, C_sioux = prepare_network_data(ema)

OD_sioux, capacities_sioux, target_flows_sioux = load_inp_outp_cap(directory_sioux)
X_sioux = np.concat([OD_sioux.reshape(OD_sioux.shape[0], -1), capacities_sioux.reshape(capacities_sioux.shape[0], -1)], axis=1)
nonzero_indices = np.nonzero(target_flows_sioux[0])
y_sioux = target_flows_sioux[:, nonzero_indices[0], nonzero_indices[1]]
OD_ema, capacities_ema, target_flows_ema = load_inp_outp_cap(directory_ema)
X_ema = np.concat([OD_ema.reshape(OD_ema.shape[0], -1), capacities_ema.reshape(capacities_ema.shape[0], -1)], axis=1)
nonzero_indices = np.nonzero(target_flows_ema[0])
y_ema = target_flows_ema[:, nonzero_indices[0], nonzero_indices[1]]

In [ ]:
sioux = create_network_df(network_name="SiouxFalls")
T_0_sioux, C_sioux = prepare_network_data(sioux)

ema = create_network_df(network_name="EMA")
T_0_sioux, C_sioux = prepare_network_data(ema)

OD_sioux, capacities_sioux, target_flows_sioux = load_inp_outp_cap(directory_sioux)
X_sioux = OD_sioux.reshape(OD_sioux.shape[0], -1)
nonzero_indices = np.nonzero(target_flows_sioux[0])
y_sioux = target_flows_sioux[:, nonzero_indices[0], nonzero_indices[1]]
OD_ema, capacities_ema, target_flows_ema = load_inp_outp_cap(directory_ema)
X_ema = OD_ema.reshape(OD_ema.shape[0], -1)
nonzero_indices = np.nonzero(target_flows_ema[0])
y_ema = target_flows_ema[:, nonzero_indices[0], nonzero_indices[1]]

In [ ]:
X_shape

In [16]:
scaler_X_sioux = StandardScaler()
X_scaled_sioux = scaler_X_sioux.fit_transform(X_sioux)

scaler_y_sioux = StandardScaler()
y_scaled_sioux = scaler_y_sioux.fit_transform(y_sioux)

X_train_sioux, X_test_sioux, y_train_sioux, y_test_sioux = train_test_split(X_scaled_sioux, y_scaled_sioux, test_size=0.1, random_state=42)
model_sioux = LinearRegression()
model_sioux.fit(X_train_sioux, y_train_sioux)

LinearRegression()

In [ ]:
scaler_X_ema = StandardScaler()
X_scaled_ema = scaler_X_ema.fit_transform(X_ema)

scaler_y_ema = StandardScaler()
y_scaled_ema = scaler_y_ema.fit_transform(y_ema)

X_train_ema, X_test_ema, y_train_ema, y_test_ema = train_test_split(X_scaled_ema, y_scaled_ema, test_size=0.1, random_state=42, shuffle=False)
model_ema = LinearRegression()
model_ema.fit(X_train_ema, y_train_ema)

LinearRegression()

In [ ]:
y_pred_scaled_sioux = model_sioux.predict(X_test_sioux)
y_pred_scaled_ema = model_ema.predict(X_test_ema)
y_pred_sioux = scaler_y_sioux.inverse_transform(y_pred_scaled_sioux)
y_test_original_sioux = scaler_y_sioux.inverse_transform(y_test_sioux)
y_pred_ema = scaler_y_ema.inverse_transform(y_pred_scaled_ema)
y_test_original_ema = scaler_y_ema.inverse_transform(y_test_ema)

In [ ]:
metrics_sioux = evaluate(y_test_original_sioux, y_pred_sioux)
metrics_ema = evaluate(y_test_original_ema, y_pred_ema)

In [ ]:
y_test_original_sioux

array([[ 5698.,  9658.,  5450., ..., 11779.,  9140.,  8331.],
       [ 4987.,  8796.,  4546., ..., 10771.,  9954.,  6537.],
       [ 4890.,  7441.,  4632., ..., 10595., 10989.,  7032.],
       ...,
       [ 5914.,  7919.,  4338., ..., 10531.,  8441.,  7371.],
       [ 5333.,  9864.,  5353., ..., 10486., 10195.,  8589.],
       [ 5549.,  8739.,  4928., ..., 11025.,  9951.,  8138.]],
      shape=(500, 76))

In [ ]:
y_pred_sioux

array([[ 5543.44830125, 10782.95806047,  6309.07931478, ...,
        11495.30030244, 10241.99372872,  8389.0548898 ],
       [ 5112.12616358,  8876.4807196 ,  4808.03882583, ...,
        10170.46155026,  9948.11138886,  7158.99430656],
       [ 5828.97901185,  8090.43211387,  5312.03945385, ...,
        10574.5455695 ,  9995.76139744,  7473.09884554],
       ...,
       [ 4982.36446321,  8486.22595091,  4687.89690115, ...,
        11170.77169136,  9173.47348658,  7448.66874654],
       [ 5453.98819927,  9826.51333387,  5226.59961191, ...,
        10905.92210796,  9712.78368923,  8414.80845365],
       [ 5803.2181026 ,  9177.26321062,  5569.42939035, ...,
        10561.15131146, 10300.95771554,  8128.98984488]], shape=(500, 76))

In [ ]:
metrics_sioux

{'MSE': np.float64(337505.3835291807),
 'RMSE': np.float64(580.9521353168269),
 'MAE': np.float64(457.3119946624183),
 'R2': np.float64(0.9859530436376585),
 'MAPE': np.float64(0.04279460928590491),
 'MedianAE': np.float64(454.1782953332592)}

In [ ]:
metrics_ema

{'MSE': np.float64(804.2135487656177),
 'RMSE': np.float64(28.358659149642772),
 'MAE': np.float64(10.97151680730338),
 'R2': np.float64(0.9995044284884946),
 'MAPE': np.float64(0.013388953400777967),
 'MedianAE': np.float64(10.413788364734412)}

In [ ]:
metrics_ema

{'MSE': np.float64(7509.2846990838225),
 'RMSE': np.float64(86.65612903357628),
 'MAE': np.float64(33.818753113218946),
 'R2': np.float64(0.995023869983755),
 'MAPE': np.float64(0.07908205964749586),
 'MedianAE': np.float64(32.96242214374281)}

# KANs

In [56]:
from efficient_kan import KAN

In [110]:
target_flows_sioux[10].mean()

np.float64(442.08506944444446)

In [116]:
target_flows_sioux

array([[[   0, 1261, 1872, ...,    0,    0,    0],
        [ 892,    0,    0, ...,    0,    0,    0],
        [1602,    0,    0, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0, 3298,    0],
        [   0,    0,    0, ..., 3417,    0, 1890],
        [   0,    0,    0, ...,    0, 1849,    0]],

       [[   0, 5026, 7928, ...,    0,    0,    0],
        [4365,    0,    0, ...,    0,    0,    0],
        [8014,    0,    0, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0, 8699,    0],
        [   0,    0,    0, ..., 9746,    0, 8402],
        [   0,    0,    0, ...,    0, 6427,    0]],

       [[   0, 1460, 1781, ...,    0,    0,    0],
        [1181,    0,    0, ...,    0,    0,    0],
        [1824,    0,    0, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0, 3170,    0],
        [   0,    0,    0, ..., 2965,    0, 1739],
        [   0,    0,    0, ...,    0, 1884,    0]],

       ...,

       [[   0, 1127, 173

In [115]:
scaler_y_sioux.inverse_transform(y_train_sioux)[100]

array([1101., 1779., 1137., 1943., 1565., 3254., 1940., 2925., 4292.,
       2050., 4158., 3870., 2198., 2107., 3630., 4879., 2615., 3979.,
       5058., 2885.,  547., 4191., 2641.,  513., 4975., 4814., 5988.,
       3998., 5850.,  309., 2644., 5845., 3135., 4641., 2195., 2791.,
       3512., 4447., 3305., 5104., 2963., 2610., 4233., 2399., 6806.,
       6120., 3744., 6255., 6528., 3352., 1185., 5434., 4745., 3717.,
       3848., 3272., 6282., 5853., 1417., 3512., 1785., 1813., 2024.,
       1524., 4447., 4696., 7190., 1952., 4250., 3194., 2580., 3276.,
       1914., 4339., 4087., 1280.])

In [111]:
target_flows_sioux.mean()

np.float64(507.78454774305555)

In [117]:
from efficient_kan import KAN
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.FloatTensor(X_train_sioux)
y_train_tensor = torch.FloatTensor(y_train_sioux)
X_test_tensor = torch.FloatTensor(X_test_sioux)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model_kan_2 = KAN(
    layers_hidden=[X_sioux.shape[1], y_sioux.shape[1]],
    spline_order=5,
    grid_eps=0.01
)

optimizer = optim.AdamW(model_kan_2.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-5)

for epoch in range(200):
    model_kan_2.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan_2(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step(total_loss/len(train_loader))
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

/home/polina/kans/kan_venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:198: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1, Loss: 0.1640
Epoch 2, Loss: 0.0843
Epoch 3, Loss: 0.0468
Epoch 4, Loss: 0.0287
Epoch 5, Loss: 0.0174
Epoch 6, Loss: 0.0139
Epoch 7, Loss: 0.0111
Epoch 8, Loss: 0.0091
Epoch 9, Loss: 0.0075
Epoch 10, Loss: 0.0075
Epoch 11, Loss: 0.0063
Epoch 12, Loss: 0.0116
Epoch 13, Loss: 0.0084
Epoch 14, Loss: 0.0100
Epoch 15, Loss: 0.0124
Epoch 16, Loss: 0.0106
Epoch 17, Loss: 0.0244
Epoch 18, Loss: 0.0136
Epoch 19, Loss: 0.0131
Epoch 20, Loss: 0.0177
Epoch 21, Loss: 0.0121
Epoch 22, Loss: 0.0218
Epoch 23, Loss: 0.0632
Epoch 24, Loss: 0.0337
Epoch 25, Loss: 0.0128
Epoch 26, Loss: 0.0109
Epoch 27, Loss: 0.0191
Epoch 28, Loss: 0.0098
Epoch 29, Loss: 0.0058
Epoch 30, Loss: 0.0043
Epoch 31, Loss: 0.0691
Epoch 32, Loss: 0.0432
Epoch 33, Loss: 0.0101
Epoch 34, Loss: 0.0064
Epoch 35, Loss: 0.0045
Epoch 36, Loss: 0.0036
Epoch 37, Loss: 0.0034
Epoch 38, Loss: 0.0065
Epoch 39, Loss: 0.0079
Epoch 40, Loss: 0.0171
Epoch 41, Loss: 0.0305
Epoch 42, Loss: 0.0172
Epoch 43, Loss: 0.0165
Epoch 44, Loss: 0.01

In [ ]:
from efficient_kan import KAN
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.FloatTensor(X_train_sioux)
y_train_tensor = torch.FloatTensor(y_train_sioux)
X_test_tensor = torch.FloatTensor(X_test_sioux)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model_kan_2 = KAN(
    layers_hidden=[X_sioux.shape[1], 128, y_sioux.shape[1]],
    spline_order=10,
    grid_eps=0.01
)

optimizer = optim.AdamW(model_kan_2.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()

for epoch in range(200):
    model_kan_2.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan_2(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.8495
Epoch 2, Loss: 0.3643
Epoch 3, Loss: 0.1678
Epoch 4, Loss: 0.0969
Epoch 5, Loss: 0.0661
Epoch 6, Loss: 0.0508
Epoch 7, Loss: 0.0426
Epoch 8, Loss: 0.0371
Epoch 9, Loss: 0.0339
Epoch 10, Loss: 0.0309
Epoch 11, Loss: 0.0292
Epoch 12, Loss: 0.0276
Epoch 13, Loss: 0.0262
Epoch 14, Loss: 0.0249
Epoch 15, Loss: 0.0235
Epoch 16, Loss: 0.0223
Epoch 17, Loss: 0.0213
Epoch 18, Loss: 0.0205
Epoch 19, Loss: 0.0193
Epoch 20, Loss: 0.0184
Epoch 21, Loss: 0.0175
Epoch 22, Loss: 0.0164
Epoch 23, Loss: 0.0151
Epoch 24, Loss: 0.0143
Epoch 25, Loss: 0.0133
Epoch 26, Loss: 0.0127
Epoch 27, Loss: 0.0115
Epoch 28, Loss: 0.0106
Epoch 29, Loss: 0.0099
Epoch 30, Loss: 0.0096
Epoch 31, Loss: 0.0091
Epoch 32, Loss: 0.0085
Epoch 33, Loss: 0.0078
Epoch 34, Loss: 0.0076
Epoch 35, Loss: 0.0071
Epoch 36, Loss: 0.0069
Epoch 37, Loss: 0.0066
Epoch 38, Loss: 0.0063
Epoch 39, Loss: 0.0060
Epoch 40, Loss: 0.0057
Epoch 41, Loss: 0.0056
Epoch 42, Loss: 0.0052
Epoch 43, Loss: 0.0050
Epoch 44, Loss: 0.00

In [ ]:
model_kan_2.eval()
with torch.no_grad():
    y_pred_scaled = model_kan_2(X_test_tensor).numpy()

y_pred = scaler_y_sioux.inverse_transform(y_pred_scaled)
y_true = scaler_y_sioux.inverse_transform(y_test_sioux)

metrics = evaluate(y_true, y_pred)

In [ ]:
y_true

array([[1311., 1596., 1360., ..., 3538., 3956., 1498.],
       [1251., 1785., 1034., ..., 3183., 4159., 2000.],
       [1152., 1814.,  925., ..., 3773., 4255., 1714.],
       ...,
       [1060., 2161., 1206., ..., 3117., 4102., 1896.],
       [ 921., 2186., 1241., ..., 2867., 4533., 1820.],
       [1170., 1963., 1101., ..., 3513., 4199., 1652.]], shape=(431, 76))

In [ ]:
y_pred

array([[1324.1625 , 1580.8761 , 1351.5295 , ..., 3521.4397 , 3985.8433 ,
        1481.2412 ],
       [1232.4254 , 1760.6111 , 1032.7969 , ..., 3226.3218 , 4102.511  ,
        1999.0665 ],
       [1158.4594 , 1819.7783 ,  931.97406, ..., 3799.7368 , 4227.034  ,
        1674.4282 ],
       ...,
       [1078.95   , 2130.396  , 1203.9078 , ..., 3274.2412 , 4217.5845 ,
        1906.9323 ],
       [ 928.80414, 2181.9243 , 1244.382  , ..., 2874.565  , 4520.871  ,
        1800.3038 ],
       [1174.824  , 1985.928  , 1103.1263 , ..., 3642.1396 , 4167.1562 ,
        1663.945  ]], shape=(431, 76), dtype=float32)

In [ ]:
metrics

{'MSE': np.float64(8541.329203132515),
 'RMSE': np.float64(92.4193118516499),
 'MAE': np.float64(63.466872423801455),
 'R2': np.float64(0.9967542253649497),
 'MAPE': np.float64(0.03699373078704261),
 'MedianAE': np.float64(62.692831340589024)}

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_sioux)
y_train_tensor = torch.FloatTensor(y_train_sioux)
X_test_tensor = torch.FloatTensor(X_test_sioux)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

model_kan_2 = KAN(
    layers_hidden=[X_sioux.shape[1], X_sioux.shape[1], y_sioux.shape[1]],
    spline_order=5,
    grid_eps=0.01
)

optimizer = optim.AdamW(model_kan_2.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()

for epoch in range(100):
    model_kan_2.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan_2(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.6741
Epoch 2, Loss: 0.1850
Epoch 3, Loss: 0.1007
Epoch 4, Loss: 0.0645
Epoch 5, Loss: 0.0468
Epoch 6, Loss: 0.0374
Epoch 7, Loss: 0.0310
Epoch 8, Loss: 0.0260
Epoch 9, Loss: 0.0219
Epoch 10, Loss: 0.0174
Epoch 11, Loss: 0.0139
Epoch 12, Loss: 0.0107
Epoch 13, Loss: 0.0083
Epoch 14, Loss: 0.0064
Epoch 15, Loss: 0.0054
Epoch 16, Loss: 0.0046
Epoch 17, Loss: 0.0043
Epoch 18, Loss: 0.0038
Epoch 19, Loss: 0.0036
Epoch 20, Loss: 0.0034
Epoch 21, Loss: 0.0035
Epoch 22, Loss: 0.0041
Epoch 23, Loss: 0.0046
Epoch 24, Loss: 0.0051
Epoch 25, Loss: 0.0047
Epoch 26, Loss: 0.0050
Epoch 27, Loss: 0.0051
Epoch 28, Loss: 0.0050
Epoch 29, Loss: 0.0054
Epoch 30, Loss: 0.0052
Epoch 31, Loss: 0.0052
Epoch 32, Loss: 0.0053
Epoch 33, Loss: 0.0054
Epoch 34, Loss: 0.0058
Epoch 35, Loss: 0.0055
Epoch 36, Loss: 0.0049
Epoch 37, Loss: 0.0053
Epoch 38, Loss: 0.0053
Epoch 39, Loss: 0.0053
Epoch 40, Loss: 0.0053
Epoch 41, Loss: 0.0050
Epoch 42, Loss: 0.0050
Epoch 43, Loss: 0.0048
Epoch 44, Loss: 0.00

In [ ]:
model_kan_2.eval()
with torch.no_grad():
    y_pred_scaled = model_kan_2(X_test_tensor).numpy()

y_pred = scaler_y_sioux.inverse_transform(y_pred_scaled)
y_true = scaler_y_sioux.inverse_transform(y_test_sioux)

metrics = evaluate(y_true, y_pred)

In [ ]:
metrics

{'MSE': np.float64(7649.034706194849),
 'RMSE': np.float64(87.45876003119898),
 'MAE': np.float64(62.710075544253655),
 'R2': np.float64(0.997109487068429),
 'MAPE': np.float64(0.03425301005540989),
 'MedianAE': np.float64(61.7509984468159)}

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_sioux)
y_train_tensor = torch.FloatTensor(y_train_sioux)
X_test_tensor = torch.FloatTensor(X_test_sioux)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model_kan_3 = KAN(
    layers_hidden=[X_sioux.shape[1], X_sioux.shape[1] + 5, y_sioux.shape[1]],
    spline_order=5,
    grid_eps=0.01
)

optimizer = optim.AdamW(model_kan_3.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()

for epoch in range(60):
    model_kan_3.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan_3(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.1229
Epoch 2, Loss: 0.0513
Epoch 3, Loss: 0.0288
Epoch 4, Loss: 0.0191
Epoch 5, Loss: 0.0141
Epoch 6, Loss: 0.0119
Epoch 7, Loss: 0.0103
Epoch 8, Loss: 0.0091
Epoch 9, Loss: 0.0086
Epoch 10, Loss: 0.0107
Epoch 11, Loss: 0.0118
Epoch 12, Loss: 0.0130
Epoch 13, Loss: 0.0128
Epoch 14, Loss: 0.0080
Epoch 15, Loss: 0.0067
Epoch 16, Loss: 0.0061
Epoch 17, Loss: 0.0048
Epoch 18, Loss: 0.0046
Epoch 19, Loss: 0.0048
Epoch 20, Loss: 0.0039
Epoch 21, Loss: 0.0028
Epoch 22, Loss: 0.0028
Epoch 23, Loss: 0.0024
Epoch 24, Loss: 0.0020
Epoch 25, Loss: 0.0018
Epoch 26, Loss: 0.0018
Epoch 27, Loss: 0.0015
Epoch 28, Loss: 0.0023
Epoch 29, Loss: 0.0024
Epoch 30, Loss: 0.0019
Epoch 31, Loss: 0.0021
Epoch 32, Loss: 0.0016
Epoch 33, Loss: 0.0012
Epoch 34, Loss: 0.0014
Epoch 35, Loss: 0.0010
Epoch 36, Loss: 0.0008
Epoch 37, Loss: 0.0007
Epoch 38, Loss: 0.0007
Epoch 39, Loss: 0.0008
Epoch 40, Loss: 0.0015
Epoch 41, Loss: 0.0078
Epoch 42, Loss: 0.0039
Epoch 43, Loss: 0.0040
Epoch 44, Loss: 0.00

In [ ]:
model_kan_3.eval()
with torch.no_grad():
    y_pred_scaled = model_kan_3(X_test_tensor).numpy()

y_pred = scaler_y_sioux.inverse_transform(y_pred_scaled)
y_true = scaler_y_sioux.inverse_transform(y_test_sioux)

metrics = evaluate(y_true, y_pred)

In [ ]:
metrics

{'MSE': np.float64(5907.968971868564),
 'RMSE': np.float64(76.8633135628992),
 'MAE': np.float64(53.191121329720595),
 'R2': np.float64(0.9977524206681404),
 'MAPE': np.float64(0.028331366693571915),
 'MedianAE': np.float64(51.69655247738487)}

In [ ]:
from kan import 

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_ema)
y_train_tensor = torch.FloatTensor(y_train_ema)
X_test_tensor = torch.FloatTensor(X_test_ema)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

model_kan_ema = KAN(
    layers_hidden=[X_ema.shape[1], 128, y_ema.shape[1]],
    spline_order=5,
    grid_eps=0.01
)

optimizer = optim.AdamW(model_kan_ema.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()

for epoch in range(60):
    model_kan_ema.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan_ema(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.7309


KeyboardInterrupt: 